Produce HTML with condensed prompt with all fields

In [ ]:
import re
import json
import time
import os
from dotenv import load_dotenv

import pandas as pd # type: ignore
from pandas import DataFrame # type: ignore
from openai import OpenAI # type: ignore
from rouge_score import rouge_scorer # type: ignore
from openpyxl import Workbook # type: ignore
from openpyxl.utils.dataframe import dataframe_to_rows # type: ignore
from openpyxl.styles import Alignment # type: ignore
from openpyxl.utils import get_column_letter # type: ignore

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

ASSISTANT_ID = "asst_ybGhVb3VbLVinE8r5NXXxKT5" 

def clear_all_dataframes():
    for name in list(globals()):
        if isinstance(globals()[name], pd.DataFrame):
            del globals()[name]
clear_all_dataframes() # type: ignore

def read_prompt_csv() -> DataFrame:
    df_prompt = pd.read_csv('Data/Input-Output/df_scored_in.csv')
    return df_prompt
df_prompt = read_prompt_csv() # type: ignore

def subset_df_prompt(df_prompt: DataFrame) -> DataFrame:
    df_prompt_subset = df_prompt[['field','prompt_description']]
    return df_prompt_subset
df_prompt_subset = subset_df_prompt(df_prompt)

def convert_df_prompt_to_prompt_str(df_prompt: DataFrame) -> str:
    prompt_json_dict = {row['field']: row['prompt_description'] 
                        for index, row in df_prompt.iterrows()}
    prompt = json.dumps(prompt_json_dict, indent=4)
    return prompt
prompt = convert_df_prompt_to_prompt_str(df_prompt_subset) # type: ignore 

def write_str_to_txt_file(str_object, file_path):
    with open(file_path, 'w') as file:
        file.write(str_object)
    return None
write_str_to_txt_file(prompt,"Data/Prompt/input_prompt.txt") # for debug -- it's a log

def run_assistant_code(prompt: str) -> DataFrame:
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": (f"Extract the values for each column and return a json format. \n"
                           f"return only the json format without additional words or context: {prompt}")
            },
        ]
    )

    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=ASSISTANT_ID)

    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(1)

    message_response = client.beta.threads.messages.list(thread_id=thread.id)
    messages = message_response.data
    return messages
messages = run_assistant_code(prompt) # type: ignore -- also couple of error their.

def parse_assistant_output_into_dict(messages):
    def extract_json(text):
        # Regular expression pattern to find JSON starting with `{` and ending with `}`
        pattern = r'\{.*\}'
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(0)  # Return the matched JSON string
        return None  # Return None if no JSON is found

    latest_message = messages[0]
    json_list = [block.text.value for block in latest_message.content]
    cleaned_json_strings = [extract_json(text) for text in json_list if extract_json(text) is not None]
    dictionaries = [json.loads(jstring) for jstring in cleaned_json_strings] 
    return dictionaries
dictionaries = parse_assistant_output_into_dict(messages)

def parse_assistant_output_dict_into_df(dictionaries):
    df_chatgpt_values = pd.DataFrame(dictionaries)
    df_chatgpt_values_transposed = df_chatgpt_values.transpose()
    df_chatgpt_values_transposed.reset_index(inplace=True)
    df_chatgpt_values_transposed.columns = ['field', 'chatgpt_value']
    return df_chatgpt_values_transposed
df_chatgpt = parse_assistant_output_dict_into_df(dictionaries)

def replace_values_no_information_string_to_nan(df_chatgpt: DataFrame):
    values_to_replace = re.compile(r'\b(no|not specified|not provided|not explicit|in the document)\b', re.IGNORECASE)
    return df_chatgpt['chatgpt_value'].replace(values_to_replace,pd.NA)
df_chatgpt['chatgpt_value'] = replace_values_no_information_string_to_nan(df_chatgpt)

def retrieve_category(df_chatgpt: DataFrame) -> DataFrame:
    df_field_category = pd.read_csv('Data/Supplemental/field_category.csv')
    df_chatgpt_category = pd.merge(df_chatgpt, df_field_category, on='field')
    return df_chatgpt_category
df_chatgpt_category = retrieve_category(df_chatgpt)

def generate_html(df_chatgpt_category: DataFrame):
    html_output = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="UTF-8">
    <title>Opportunity Information</title>
    <style>
        body { font-family: Arial, sans-serif; background: #fff; color: #333; }
        .category { margin-bottom: 20px; }
        .header { background: #007BFF; padding: 10px; cursor: pointer; color: #fff; }
        .content { border: 1px solid #ccc; border-top: none; padding: 10px; display: flex; flex-wrap: wrap; }
        .column { flex: 50%; }
        .field { box-sizing: border-box; padding: 5px; }
        .field-name { font-weight: bold; }
        img.logo { height: 50px; margin-bottom: 20px; display: block; }
    </style>
    <script>
        document.addEventListener("DOMContentreaded", function() {
            var headers = document.querySelectorAll('.header');
            headers.forEach(function(header) {
                header.addEventListener('click', function() {
                    var content = this.nextElementSibling;
                    content.style.display = content.style.display === 'flex' ? 'none' : 'flex';
                });
            });
        });
    </script>
    </head>
    <body>

    <img src="buscom_logo.jpeg" class="logo" alt="Bus.com Logo">
    """

    # Define the order of categories
    category_order = [
        "Opportunity Information",
        "High Level Summary",
        "Dates and Deadlines",
        "Opportunity Team & Contacts",
        "Estimates",
        "RFP Summary",
        "Deal Specifics",
        "Additional Information",
        "Lost Opportunity Analysis"
    ]

    # Define specific field orderings for complex categories
    dates_fields_left = ["Posted Date", "Deadline for Questions", "Prebid Attendance", "Pre-Bid Conference",
                        "Proposal Deadline Date", "Interview Information", "Last Addendum Date"]
    dates_fields_right = ["Event Start Date", "Event End Date", "Contract Duration (Months)",
                        "Contract Extension Term", "Options to Renew"]
    contacts_fields = ["Contact: Title", "Contact: Email", "Contact: Phone"]
    estimates_left = ["Estimated Contract Value", "Estimated Supplier Amount"]
    estimates_right = ["Estimated Cost", "Custom Payment Terms"]
    rfp_summary_left = ["Region Area Name", "Region Area Kind", "RFP Name", "RFP Service Type", "Submission Type", "Award Type"]
    rfp_summary_right = ["Subcontracting Allowed", "Supplier Partner", "Incumbent Supplier", "Pricing Type", "Proposal Writer"]
    deal_specifics_left = ["Number of Buses", "Facility Provided by Agency", "Maintenance Provided by Agency", "Parking Provided by Agency", "Living Wage Requirements", "Disadvantaged Business Requirements", "Fleet Requirement", "Fuel Provided By", "Vehicle Fuel Type", "Technology Needed", "Technology Partner", "Bidder References Required", "Liquidated Damages"]
    deal_specifics_right = ["Number of Awarded Vendors", "Facility Notes", "Maintenance Notes", "Parking Notes", "Living Wage Comments", "Disadvantaged Business Comments", "Fleet Requirement Comments", "Fuel Notes", "Vehicle Provider", "Technology Description", "Bidder References Description", "Liquidated Damage Information"]

    # Loop through the predefined list of categories
    for category in category_order:
        if category in df_chatgpt_category['category'].values:
            filtered_df = df_chatgpt_category[df_chatgpt_category['category'] == category]
            html_output += f'<div class="category">\n    <div class="header">{category}</div>\n    <div class="content" style="display: flex;">'

            if category == "Dates and Deadlines":
                # Specific handling for Dates and Deadlines with left and right columns
                html_output += '<div class="column">\n'
                for field in dates_fields_left:
                    if field in filtered_df['field'].values:
                        value = filtered_df.loc[filtered_df['field'] == field, 'chatgpt_value'].iloc[0]
                        html_output += f'        <div class="field"><span class="field-name">{field}:</span> {value}</div>\n'
                html_output += '</div>\n<div class="column">\n'
                for field in dates_fields_right:
                    if field in filtered_df['field'].values:
                        value = filtered_df.loc[filtered_df['field'] == field, 'chatgpt_value'].iloc[0]
                        html_output += f'        <div class="field"><span class="field-name">{field}:</span> {value}</div>\n'
                html_output += '</div>\n'

            elif category == "Opportunity Team & Contacts":
                # Vertical display for contact fields
                for field in contacts_fields:
                    if field in filtered_df['field'].values:
                        value = filtered_df.loc[filtered_df['field'] == field, 'chatgpt_value'].iloc[0]
                        html_output += f'        <div class="field" style="flex: 100%;"><span class="field-name">{field}:</span> {value}</div>\n'

            elif category == "Estimates":
                # Split fields into two columns for Estimates
                html_output += '<div class="column">\n'
                for field in estimates_left:
                    if field in filtered_df['field'].values:
                        value = filtered_df.loc[filtered_df['field'] == field, 'chatgpt_value'].iloc[0]
                        html_output += f'        <div class="field"><span class="field-name">{field}:</span> {value}</div>\n'
                html_output += '</div>\n<div class="column">\n'
                for field in estimates_right:
                    if field in filtered_df['field'].values:
                        value = filtered_df.loc[filtered_df['field'] == field, 'chatgpt_value'].iloc[0]
                        html_output += f'        <div class="field"><span class="field-name">{field}:</span> {value}</div>\n'
                html_output += '</div>\n'

            elif category == "RFP Summary" or category == "Deal Specifics":
                # Handle RFP Summary and Deal Specifics with custom field orders
                html_output += '<div class="column">\n'
                fields_left = rfp_summary_left if category == "RFP Summary" else deal_specifics_left
                for field in fields_left:
                    if field in filtered_df['field'].values:
                        value = filtered_df.loc[filtered_df['field'] == field, 'chatgpt_value'].iloc[0]
                        html_output += f'        <div class="field"><span class="field-name">{field}:</span> {value}</div>\n'
                html_output += '</div>\n<div class="column">\n'
                fields_right = rfp_summary_right if category == "RFP Summary" else deal_specifics_right
                for field in fields_right:
                    if field in filtered_df['field'].values:
                        value = filtered_df.loc[filtered_df['field'] == field, 'chatgpt_value'].iloc[0]
                        html_output += f'        <div class="field"><span class="field-name">{field}:</span> {value}</div>\n'
                html_output += '</div>\n'

            elif category == "Additional Information":
                # Vertical display of all fields for Additional Information
                for index, row in filtered_df.iterrows():
                    html_output += f'        <div class="field" style="flex: 100%;"><span class="field-name">{row["field"]}:</span> {row["chatgpt_value"]}</div>\n'

            else:
                # Normal field handling for other categories
                for index, row in filtered_df.iterrows():
                    html_output += f'        <div class="field"><span class="field-name">{row["field"]}:</span> {row["chatgpt_value"]}</div>\n'

            html_output += '    </div>\n</div>\n'

    html_output += """
        </body>
        </html>
    """
    return html_output
html_content = generate_html(df_chatgpt_category)

def save_html(html_content):
    html_file_path = "HTML-CSS/rfp_summary.html"
    with open(html_file_path, 'w') as file:
        file.write(html_content)
    return None
save_html(html_content)

def retrieve_prompt_description(df_chatgpt_category, df_prompt):
    df_chatgpt_category_prompt_description = pd.merge(df_chatgpt_category, 
                             df_prompt[['field', 'prompt_description']]
                             , on='field'
                             , how='left')
    return df_chatgpt_category_prompt_description    
df_chatgpt_category_prompt_description = retrieve_prompt_description(df_chatgpt_category, df_prompt)

def retrieve_score_type(df_chatgpt_category_prompt_description):
    df_score_type = pd.read_csv('Data/Supplemental/field_score_type.csv')
    df_chatgpt_category_prompt_description_score = pd.merge(df_chatgpt_category_prompt_description, 
                             df_score_type,
                              on='field',
                              how='left')
    return df_chatgpt_category_prompt_description_score    
df_chatgpt_category_prompt_description_score = retrieve_score_type(df_chatgpt_category_prompt_description)

def read_sf_csv() -> DataFrame:
    try:
        df_sf = pd.read_csv('Data/Test/test_city_of_turlock.csv')
        return df_sf
    except Exception as e:
        raise Exception(f"Failed to read data: {e}")
df_sf = read_sf_csv() # type: ignore

def merge_df_test_with_df_chatgpt(df_sf: DataFrame,df_chatgpt_category_prompt_description_score: DataFrame):
    df_sf_chatgpt = pd.merge(df_sf, 
                             df_chatgpt_category_prompt_description_score[['field', 'chatgpt_value','prompt_description','field_score_type']]
                             , on='field'
                             , how='right')
    return df_sf_chatgpt
df_sf_chatgpt = merge_df_test_with_df_chatgpt(df_sf,df_chatgpt_category_prompt_description_score)

def reorder_df_sf_chatgpt_column(df_sf_chatgpt: DataFrame):
    new_order = ['field', 'prompt_description','field_score_type','chatgpt_value','sf_value']
    return df_sf_chatgpt[new_order]
df_sf_chatgpt = reorder_df_sf_chatgpt_column(df_sf_chatgpt) 

def standardize_missing_values(df):
    for column in df.columns:
        df[column] = df[column].map(lambda x: pd.NA if pd.isna(x) else x)
    return df
df_sf_chatgpt = standardize_missing_values(df_sf_chatgpt) 

def score_df(df_sf_chatgpt: DataFrame) -> DataFrame:
    field_score_types = ['binary', 'human']
    processed_dfs = []
    
    for field_score_type in field_score_types:
        processed_df = process_scores(df_sf_chatgpt, field_score_type)
        processed_dfs.append(processed_df)
    
    df_sf_chatgpt_score_type_prepared_score = pd.concat(processed_dfs, axis=0).drop_duplicates()
    df_sf_chatgpt_score_type_prepared_score['score'] = df_sf_chatgpt_score_type_prepared_score['score'].round(2)

    return df_sf_chatgpt_score_type_prepared_score
def process_scores(df, score_type):
    """Process scores based on field_score_type, filtering data to relevant rows."""
    df_filtered = df[df['field_score_type'] == score_type].copy()
    if score_type == 'binary':
        return calculate_binary_score(df_filtered, 'sf_value', 'chatgpt_value')
    elif score_type == 'human':
        return calculate_rouge_l(df_filtered, 'sf_value', 'chatgpt_value')
    else:
        raise ValueError("Invalid score_type provided. Choose 'binary' or 'human'.")
def calculate_binary_score(df, col1, col2):
    """Calculates binary score comparing two column values where pd.NA values are considered equal."""
    def custom_na_comparison(x, y):
        if pd.isna(x) and pd.isna(y):
            return 1  
        elif pd.isna(x) or pd.isna(y):
            return 0 
        elif x == y:
            return 1 
        else:
            return 0 
    # Apply custom comparison across the columns
    df['score'] = [custom_na_comparison(x, y) for x, y in zip(df[col1], df[col2])]
    return df
def calculate_rouge_l(df, col1, col2):
    """Applies ROUGE-L score calculation to DataFrame."""
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    df['score'] = df.apply(lambda row: scorer.score(str(row[col1])
                                                    , str(row[col2]))['rougeL'].fmeasure
                                                    , axis=1)
    return df
df_sf_chatgpt_score = score_df(df_sf_chatgpt)   			

df_sf_chatgpt_score.to_csv('Data/Input-Output/df_scored_out.csv')

def write_to_excel(df: DataFrame):
    wb = Workbook()
    ws = wb.active
    df = df.where(pd.notna(df), None)

    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            # Set both center alignment and wrap text
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=False)

    chatgpt_column_letter = get_column_letter(df.columns.get_loc('chatgpt_value') + 1)
    prompt_description_column_letter = get_column_letter(df.columns.get_loc('prompt_description') + 1)
    sf_value_column_letter = get_column_letter(df.columns.get_loc('sf_value') + 1)

    ws.column_dimensions[chatgpt_column_letter].width = 100
    ws.column_dimensions[prompt_description_column_letter].width = 40

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
        ws.row_dimensions[row[0].row].height = 20

    wb.save('Data/Input-Output/df_scored_out.xlsx')
write_to_excel(df_sf_chatgpt_score)

MVP: Functions to score and extract many fields, many RFPs

In [4]:
import os
from itertools import product
from pathlib import Path
import time
import re
import json

import pandas as pd
from pandas import DataFrame
from openai import OpenAI
from rouge_score import rouge_scorer
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

N_RUN = 1
# One assistant ID per RFP # asst_3ZZ33RlX4eAApKD2EskbYIsj asst_wQcnhTkLgrFfGRrHXP6QC2Ya
ASSISTANT_IDS = ["asst_wQcnhTkLgrFfGRrHXP6QC2Ya"] 

def clear_all_dataframes():
    for name in list(globals()):
        if isinstance(globals()[name], pd.DataFrame):
            del globals()[name]
clear_all_dataframes()

def read_txt_file(filename):
    file_path = Path(f"Data/Prompt/{filename}")
    content = file_path.read_text()
    return content, filename

prompt_files = ['event_start_date_prompt_v2.txt']
prompts = [read_txt_file(fname) for fname in prompt_files]
param_grid = {
    'temperature': [0.7],
    'model': ['gpt-3.5-turbo','gpt-4o-mini'],
    'prompt_details': prompts
}

def create_grid(param_grid):
    for params in product(*param_grid.values()):
        yield dict(zip(param_grid.keys(), params))

def update_assistant(assistant_id, model, temperature):
    return client.beta.assistants.update(
        assistant_id=assistant_id,
        model=model,
        temperature=temperature
    )

def run_assistant_code(assistant_id: str, prompt: str) -> DataFrame:
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                 "content": (f"Extract the values for each column and return a json format. \n"
                           f"return only the json format without additional words or context: {prompt}")
            },
        ]
    )

    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(1)

    message_response = client.beta.threads.messages.list(thread_id=thread.id)
    messages = message_response.data
    return messages
def parse_assistant_output_into_dict(messages):
    def extract_json(text):
        # Regular expression pattern to find JSON starting with `{` and ending with `}`
        pattern = r'\{.*\}'
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(0)  # Return the matched JSON string
        return None  # Return None if no JSON is found

    latest_message = messages[0]
    json_list = [block.text.value for block in latest_message.content]
    cleaned_json_strings = [extract_json(text) for text in json_list if extract_json(text) is not None]
    dictionaries = [json.loads(jstring) for jstring in cleaned_json_strings] 
    return dictionaries
def parse_assistant_output_dict_into_df(dictionaries):
    df_chatgpt_values = pd.DataFrame(dictionaries)
    df_chatgpt_values_transposed = df_chatgpt_values.transpose()
    df_chatgpt_values_transposed.reset_index(inplace=True)
    df_chatgpt_values_transposed.columns = ['field', 'chatgpt_value']
    return df_chatgpt_values_transposed

all_dfs = []
for assistant_id in ASSISTANT_IDS:
    for params in create_grid(param_grid):
        assistant = update_assistant(
            assistant_id,
            params['model'],
            params['temperature']
        )

        for i in range(N_RUN):
            prompt_content, prompt_filename = params['prompt_details']
            messages = run_assistant_code(assistant_id, prompt_content)
            dictionaries = parse_assistant_output_into_dict(messages)
            df_chatgpt_one_run = parse_assistant_output_dict_into_df(dictionaries)
            df_chatgpt_one_run = df_chatgpt_one_run.assign(
                run_number=i + 1,
                model_used=params['model'],
                temperature=params['temperature'],
                assistant_id=assistant_id,
                prompt_version=prompt_filename.split('_')[-1].split('.')[0]
            )
            print(f'Completed run {i + 1} with {params["model"]} at temperature {params["temperature"]} for {prompt_filename[:-4]} and {assistant_id}')
            all_dfs.append(df_chatgpt_one_run)

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['config'] = combined_df.apply(lambda x: f"{x['model_used']}_{x['temperature']}_{x['prompt_version']}", axis=1)

df_assistant = pd.read_csv('Data/Assistant/assistant_data.csv')
combined_df_with_assistant = pd.merge(df_assistant, 
                             combined_df[['field', 'chatgpt_value', 'run_number','config','assistant_id']],
                             on='assistant_id', 
                             how='right')
combined_df_with_assistant.rename(columns={'assistant_name': 'opportunity_name'}, inplace=True)

df_sf = pd.read_csv('Data/Test/test_both_city_of_turlock_calabases.csv')
df_simulation_sf_chatgpt = pd.merge(df_sf, 
                             combined_df_with_assistant[['field', 'chatgpt_value'
                                                         , 'run_number', 'config','opportunity_name']],
                             on=['field','opportunity_name'], 
                             how='right')

def retrieve_score_type(df_chatgpt_category_prompt_description):
    df_score_type = pd.read_csv('Data/Supplemental/field_score_type.csv')
    df_chatgpt_category_prompt_description_score = pd.merge(df_chatgpt_category_prompt_description, 
                             df_score_type,
                              on='field',
                              how='left')
    return df_chatgpt_category_prompt_description_score    
df_simulation_sf_chatgpt_score_type = retrieve_score_type(df_simulation_sf_chatgpt)

def score_df(df_sf_chatgpt: DataFrame) -> DataFrame:
    field_score_types = ['binary', 'human']
    processed_dfs = []
    
    for field_score_type in field_score_types:
        processed_df = process_scores(df_sf_chatgpt, field_score_type)
        processed_dfs.append(processed_df)
    
    df_sf_chatgpt_score_type_prepared_score = pd.concat(processed_dfs, axis=0).drop_duplicates()
    df_sf_chatgpt_score_type_prepared_score['score'] = df_sf_chatgpt_score_type_prepared_score['score'].round(2)

    return df_sf_chatgpt_score_type_prepared_score
def process_scores(df, score_type):
    """Process scores based on field_score_type, filtering data to relevant rows."""
    df_filtered = df[df['field_score_type'] == score_type].copy()
    if score_type == 'binary':
        return calculate_binary_score(df_filtered, 'sf_value', 'chatgpt_value')
    elif score_type == 'human':
        return calculate_rouge_l(df_filtered, 'sf_value', 'chatgpt_value')
    else:
        raise ValueError("Invalid score_type provided. Choose 'binary' or 'human'.")
def calculate_binary_score(df, col1, col2):
    """Calculates binary score comparing two column values where pd.NA values are considered equal."""
    def custom_na_comparison(x, y):
        if pd.isna(x) and pd.isna(y):
            return 1  
        elif pd.isna(x) or pd.isna(y):
            return 0 
        elif x == y:
            return 1 
        else:
            return 0 
    # Apply custom comparison across the columns
    df['score'] = [custom_na_comparison(x, y) for x, y in zip(df[col1], df[col2])]
    return df
def calculate_rouge_l(df, col1, col2):
    """Applies ROUGE-L score calculation to DataFrame."""
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    df['score'] = df.apply(lambda row: scorer.score(str(row[col1])
                                                    , str(row[col2]))['rougeL'].fmeasure
                                                    , axis=1)
    return df

df_simulation_runs_sf_chatgpt_scored = score_df(df_simulation_sf_chatgpt_score_type)

#df_simulation_runs_sf_chatgpt_scored.to_csv("Data/Consistency/simulation_end_date_data_only_gpt_4_turbo.csv")
df_reorder = df_simulation_runs_sf_chatgpt_scored[['opportunity_name', 'field'
                                                   , 'config', 'sf_value', 'chatgpt_value'
                                                   , 'score']]
df_reorder

Completed run 1 with gpt-3.5-turbo at temperature 0.7 for event_start_date_prompt_v2 and asst_wQcnhTkLgrFfGRrHXP6QC2Ya
Completed run 1 with gpt-4o-mini at temperature 0.7 for event_start_date_prompt_v2 and asst_wQcnhTkLgrFfGRrHXP6QC2Ya


,opportunity_name,field,config,sf_value,chatgpt_value,score
0,"City of Calabasas, CA Fixed 2024",Event Start Date,gpt-3.5-turbo_0.7_v2,2024-07-01,2022-09-01,0.0
1,"City of Calabasas, CA Fixed 2024",Event Start Date,gpt-4o-mini_0.7_v2,2024-07-01,,0.0
